In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_palette('colorblind'); sns.set_color_codes(); sns.set_style('white')
plt.style.use('classic')

import holoviews as hv
hv.notebook_extension('bokeh', 'matplotlib')
hv.plotting.mpl.MPLPlot.fig_alpha = 0   ## Bandaid: fix axisbg issue

import param
import paramnb

import numpy as np
import pandas as pd
from time import sleep

import xarray as xr
import geoviews as gv
import geoviews.feature as gf

import cartopy
from cartopy import crs as ccrs

from bokeh.tile_providers import CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TONER_BACKGROUND, STAMEN_TONER_LABELS, STAMEN_TONER
from bokeh.models import WMTSTileSource

import scipy.stats as stats
print('holoviews %s'% hv.__version__)

holoviews 1.7.0


##### Data Conversions

* Plate carrée:  https://en.wikipedia.org/wiki/Equirectangular_projection
* Mercator:      https://en.wikipedia.org/wiki/Mercator_projection

Conversions are easily carried out with [https://github.com/SciTools/cartopy](cartopy)

In [ ]:
lon=np.array([-97.70, -97.74, -97.78])
lat=np.array([ 30.29,  30.20, 30.29])

crs_m = ccrs.GOOGLE_MERCATOR
df    = pd.DataFrame(crs_m.transform_points( ccrs.PlateCarree(), lon, lat ),
                     columns=['x','y','h'])

df['x_m'],df['y_m'],_ = ccrs.Mercator().transform_points( ccrs.PlateCarree(), lon, lat )
df

In [ ]:
extent = ( df['x'].min(), df['y'].min(), df['x'].max(), df['y'].max())
print(extent)

In [2]:
def lonlat_corners_to_extents( btm_left, top_right, to=ccrs.GOOGLE_MERCATOR, src=ccrs.PlateCarree() ):
    left,  bottom = to.transform_point(btm_left [0],btm_left [1],src)
    right, top    = to.transform_point(top_right[0],top_right[1],src)
    return (left, bottom, right, top )

e=lonlat_corners_to_extents([-97.782,30.199],[-97.699,30.300])
print(e)

(-10885042.448747875, 3529155.1707309242, -10875802.931012034, 3542170.6276430194)


In [3]:
def extent_from_center( center, dx, dy, to=ccrs.GOOGLE_MERCATOR ):
    x,y = to.transform_point( center[0], center[1], ccrs.PlateCarree())
    return ( x-dx, y-dy, x+dx,y+dy )

##### Basic Maps

In [4]:
%%opts WMTS [width=350 height=250 xaxis=None yaxis=None show_grid=True]
# -------------------------------------------------------------------------
# tile sources   http://tile.stamen.com/terrain-background/zoom/{X}/{Y}.jpg
tiles = {
    'ESRI':         WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/'
                                       'World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'),
    'ArcGIS':       WMTSTileSource(url='http://server.arcgisonline.com/ArcGIS/rest/services/'
                                       'World_Street_Map/MapServer/tile/{Z}/{Y}/{X}.png' ),
    'Wikipedia':    WMTSTileSource(url='https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'),
    'OpenMap':      WMTSTileSource(url='http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png')
#    'HillShading':  WMTSTileSource(url='http://c.tiles.wmflabs.org/hillshading/{Z}/{Y}/{X}.png'),
}
tiles2 = {
    'Stamen Toner':   STAMEN_TONER,
    'Stamen Terrain': STAMEN_TERRAIN,
    'CartoDB':        CARTODBPOSITRON
}
# -------------------------------------------------------------------------
# To get a map without adding data, specify the extent and the crs
# Note that the width and height of WMTS play a role in whether the map is wrapped around
hv.NdLayout(
    { name: gv.WMTS( wmts, extents=(0,-90,360,90), crs=ccrs.PlateCarree())
            for name, wmts in tiles.items()}, kdims=['Source']).cols(2)


b':NdLayout   [Source]\n   :WMTS   [Longitude,Latitude]'

> Use Mercator crs<br><br>
computing and setting the extents results in <span style="color:red;">puzzling behaviour</span>:
*  switching between e1 and e2 in the example below results in a lesser zoom level
  for e2, even though e2 covers a smaller geographical area within e1

In [5]:
%%opts WMTS [height=250 width=300 xaxis=None yaxis=None show_grid=True]
# So only specify one dimension. Note zoom and pan works, as does reset
# Use a Mercator projection

# Austin, Texas
e1=lonlat_corners_to_extents([-97.782,30.199],[-97.699,30.300], to=ccrs.GOOGLE_MERCATOR); print("e1 =", e1)
e2=lonlat_corners_to_extents([-97.772,30.199],[-97.699,30.300], to=ccrs.GOOGLE_MERCATOR); print("e2 =", e2)
# Setting extent=e1 shows parts of Mexico

hv.NdLayout(
    { name: gv.WMTS( wmts, extents=e2, crs=ccrs.GOOGLE_MERCATOR)
            for name, wmts in tiles.items()}, kdims=['Source']).cols(2)

e1 = (-10885042.448747875, 3529155.1707309242, -10875802.931012034, 3542170.6276430194)
e2 = (-10883929.253839944, 3529155.1707309242, -10875802.931012034, 3542170.6276430194)


b':NdLayout   [Source]\n   :WMTS   [Longitude,Latitude]'

> WMTS and extents<br><br>
to be reliable, extents need to be computed
*  <span style="color:red;">AFTER</span> WMTS is instantiated,<br>
*  <span style="color:red;">USING</span> the crs of the WMTS instance ?!!

In [6]:
%%opts WMTS [height=250 width=300 xaxis=None yaxis=None shared_axes=False]

gv_map1         = gv.WMTS( tiles['Wikipedia'])
gv_map2         = gv.WMTS( tiles['Wikipedia'])

gv_map1.extents = lonlat_corners_to_extents([-97.782,30.199],[-97.699,30.300], to=gv_map1.crs)
gv_map2.extents = lonlat_corners_to_extents([-97.772,30.199],[-97.699,30.300], to=gv_map2.crs)
print("e1 =", gv_map1.extents)
print("e2 =", gv_map2.extents)

gv_map1 + gv_map2

e1 = (-10885042.448747875, 3529155.1707309242, -10875802.931012034, 3542170.6276430194)
e2 = (-10883929.253839944, 3529155.1707309242, -10875802.931012034, 3542170.6276430194)


:Layout
   .WMTS.I  :WMTS   [Longitude,Latitude]
   .WMTS.II :WMTS   [Longitude,Latitude]

> WMTS and extents<br><br>
print the proj4 parameters for confirmation
* Note that <span style='color:red;'>bokeh reset </span> does not work properly

In [7]:
%%opts WMTS [height=250 xaxis=None yaxis=None show_grid=True shared_axes=False]
# So try with just one map
# To be reliable, we need to ensure we use the same crs as the map
#       or else assign the extents AFTER we define the map
#       AND using the crs of the map
# Note bokehs reset is confused about the extent

gv_map1         = gv.WMTS( tiles['Wikipedia'])
gv_map1.extents = lonlat_corners_to_extents([-97.775,30.265],[-97.770,30.270], to=gv_map1.crs)

# Now let's try with an extent set ahead of time:
e2 = lonlat_corners_to_extents([-97.775,30.265],[-97.770,30.270], to=ccrs.GOOGLE_MERCATOR)
gv_map2         = gv.WMTS( tiles['Wikipedia'], extents=e2)

print( "Mercator proj ", ccrs.Mercator().proj4_params)
print( "map1 crs proj ", gv_map1.crs.proj4_params)
print( "map2 crs proj ", gv_map2.crs.proj4_params)
print( 'map1.extents  ', gv_map1.extents)
print( 'map2.extents  ', gv_map2.extents)
gv_map1 + gv_map2

Mercator proj  {'proj': 'merc', 'ellps': 'WGS84', 'lon_0': 0.0, 'units': 'm', 'k': 1}
map1 crs proj  {'proj': 'merc', 'a': 6378137.0, 'b': 6378137.0, 'k': 1, 'lon_0': 0.0, 'units': 'm', 'nadgrids': '@null'}
map2 crs proj  {'proj': 'merc', 'a': 6378137.0, 'b': 6378137.0, 'k': 1, 'lon_0': 0.0, 'units': 'm', 'nadgrids': '@null'}
map1.extents   (-10884263.212312324, 3537658.8054654864, -10883706.614858357, 3538303.2529758886)
map2.extents   (-10884263.212312324, 3537658.8054654864, -10883706.614858357, 3538303.2529758886)


:Layout
   .WMTS.I  :WMTS   [Longitude,Latitude]
   .WMTS.II :WMTS   [Longitude,Latitude]

##### Add data points

> <span style='color:red;'>Strange behaviour and errors</span>
* Markers such as marker='v' result in errors about orientation
* Bokeh reset sets extents to tight bounds, rather than the original

In [8]:
%%opts WMTS [height=250 xaxis=None yaxis=None show_grid=True]
%%opts Scatter( color='magenta'   size=8)
%%opts Points ( color='darkgreen' size=15 marker='d')

# This time, we do not define an extent, but we provide points instead
# We get a map area that includes the points
#    but bokeh reset will provide tight bounds
# Note: coordinate conversions require the crs of WMTS be used
#    the results are inconsistent otherwise.

lon           = np.array([-73.973599, -73.974557, -73.979744])
lat           = np.array([ 40.779770,  40.783249,  40.781216])

# ----------------------------------------------
gv_map = gv.WMTS( tiles2['CartoDB'])
# ----------------------------------------------
if False:
    # do the transformation ourselves
    points    = gv_map.crs.transform_points( ccrs.PlateCarree(), lon, lat)
    df_points = pd.DataFrame( points, columns=['x','y','h'])

    h1 = hv.Points ( gv.Dataset( df_points, kdims=['x','y'], vdims=[]))
    #h2 = hv.Scatter( gv.Dataset( df_points, kdims=['x'], vdims=['y']))
else:
    # let GeoViews handle the transformation
    h1 = gv.Points((lon, lat), crs=ccrs.PlateCarree())

gv_map*h1 #*h2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [6]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

lon           = np.array([-73.973599, -73.974557, -73.979744])
lat           = np.array([ 40.779770,  40.783249,  40.781216])

geolocator.reverse((lat[0], lon[0]))

Location(Alexander von Humboldt, West 77th Street, Central Park, Manhattan, New York County, NYC, New York, 10024, United States of America, (40.7794533, -73.9732852, 0.0))

-----
Set an extent as well as add some data points
* Note this interacts with the WMTS width setting in a non-obvious way
* Note <span style='color:red'>bokeh reset</span> is now confused in yet another way....

In [11]:
%%opts WMTS [width=300 xaxis=None yaxis=None show_grid=True]
%%opts Scatter( color='magenta'   size=8)
%%opts Points ( color='darkgreen' size=15 marker='d')

lon           = np.array([-73.973599, -73.974557, -73.979744])
lat           = np.array([ 40.779770,  40.783249,  40.781216])

# ----------------------------------------------
gv_map = gv.WMTS( tiles2['CartoDB'])
gv_map.extents = extent_from_center( (-73.974, 40.780), 500., 800., to=gv_map.crs )
# ----------------------------------------------

h1 = gv.Points ((lon, lat), crs=ccrs.PlateCarree())
#h2 = gv.Scatter(...) does not exist

gv_map*h1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

In [ ]:
%%opts Overlay [width=600 height=300] 
%%opts Points (size=0.005 cmap='viridis') [tools=['hover'] size_index=2 color_index=2 xaxis=None yaxis=None]

cities = pd.read_csv('./assets/cities.csv', encoding="ISO-8859-1")
population = gv.Dataset(cities, kdims=['City', 'Country', 'Year'])
print(cities.tail(3))

(gv.WMTS(tiles['Wikipedia']) *\
population.to(gv.Points, kdims=['Longitude', 'Latitude'],
              vdims=['Population', 'City', 'Country'], crs=ccrs.PlateCarree()))

##### Paths

In [12]:
%%opts Overlay [width=300 height=200 xaxis=None yaxis=None]
# Great Circle and PlateCarre path
#  since the crs for gv.feature is plate carree,
#  should the red path not render as a straight line?

src_dst = [[(132,-0.08), (43.17, 51.53)]]
gv.feature.land * gv.feature.borders * gv.feature.coastline *\
gv.Path(src_dst, crs=ccrs.Geodetic()) *\
gv.Path(src_dst, crs=ccrs.PlateCarree())

:Overlay
   .Land.I      :Feature   [Longitude,Latitude]
   .Borders.I   :Feature   [Longitude,Latitude]
   .Coastline.I :Feature   [Longitude,Latitude]
   .Path.I      :Path   [Longitude,Latitude]
   .Path.II     :Path   [Longitude,Latitude]

In [13]:
%%opts WMTS [width=250 xaxis=None yaxis=None show_grid=True]
%%opts Scatter( color='magenta'   size=8)
%%opts Points ( color='darkgreen' size=15 marker='d')

# Philipp Rudiger's code to subsample the great circle path
import pyproj

def get_circle_path(start, end, sampling=10000):
    sx, sy = start
    ex, ey = end
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(sx, sy, ex, ey)
    lonlats = g.npts(sx, sy, ex, ey, 1+int(dist/sampling))
    print('dist =',dist)
    lonlats.insert(0,start)
    lonlats.append(end)
    return lonlats

# ----------------------------------------------
gv_map = gv.WMTS( tiles2['CartoDB'])
# ----------------------------------------------

# Great Circle
src_dst = [[(132,-0.08), (43.17, 51.53)]]
pts = get_circle_path( src_dst[0][0], src_dst[0][1], sampling=12000)

gv_map *\
gv.Path([pts],   crs=ccrs.PlateCarree()) *\
gv.Path(src_dst, crs=ccrs.PlateCarree())

dist = 9934215.383557605


:Overlay
   .WMTS.I  :WMTS   [Longitude,Latitude]
   .Path.I  :Path   [Longitude,Latitude]
   .Path.II :Path   [Longitude,Latitude]

##### Choropleths

In [14]:
%%opts Shape [xaxis=None yaxis=None]
shapefile = './assets/boundaries/boundaries.shp'
gv.Shape.from_shapefile(shapefile, crs=ccrs.PlateCarree())

b':NdOverlay   [Index]\n   :Shape   [Longitude,Latitude]   (Level)'

In [15]:
shapes = cartopy.io.shapereader.Reader(shapefile)
referendum = pd.read_csv('./assets/referendum.csv')
referendum = hv.Dataset(referendum)
referendum.data.tail(3)

,leaveVoteshare,regionName,turnout,name,code
396,56.000000,Wales,70.199997,Newport,W06000022
397,53.700001,Wales,77.000000,Powys,W06000023
398,56.400002,Wales,67.400002,Merthyr Tydfil,W06000024


In [16]:
%%opts Shape (cmap='viridis') [tools=['hover'] width=450 height=600 colorbar=True toolbar='above' xaxis=None yaxis=None]
gv.Shape.from_records(shapes.records(), referendum, on='code', value='leaveVoteshare',
                      index='name', crs=ccrs.PlateCarree(), group='EU Referendum', drop_missing=True).redim(name='County', leaveVoteshare='Leave Vote %')

b':NdOverlay   [County]\n   :Shape   [Longitude,Latitude]   (Leave Vote %)'

##### Hexbins

In [5]:
import matplotlib.pyplot as plt
from holoviews.plotting.bokeh import PolygonPlot
from holoviews.plotting.bokeh.element import line_properties, fill_properties
# -------------------------------------------------------------------------
# Hexbins coped from @philippjfr
# -------------------------------------------------------------------------
class HexBins(hv.Points):    
    kdims = param.List([ hv.Dimension('x'), hv.Dimension('y')],
                       bounds=(2, 2))
    
    group = param.String(default='HexBins')
# -------------------------------------------------------------------------
class HexBinPlot(PolygonPlot):
    agg_func = param.Callable(default=np.sum)    
    gridsize = param.Integer(default=50)

    lod = param.Dict(default={'factor': 1, 'interval': 300,
                              'threshold': 2000, 'timeout': 500})

    style_opts = ['cmap', 'palette'] + line_properties + fill_properties
    _plot_methods = dict(single='patches')
    
    #def __init__(self, **params ):
    #    super(HexBinPlot, self).__init__( **params )

    def get_data(self, element, ranges=None, empty=False):
        if not element.vdims:
            element = element.add_dimension('z', 0, np.ones(len(element)), True)
        hxs, hys, agg = self.get_hex_data( element, gridsize=self.gridsize,
                                     agg_func=self.agg_func)
        
        data = dict(xs=list(hxs), ys=list(hys))
        mapping = dict(self._mapping)
        
        cdim = element.vdims[0]
        data[cdim.name] = agg
        style = self.style[self.cyclic_index]
        ranges = {cdim.name: (np.min(agg), np.max(agg))}
        cmapper = self._get_colormapper(cdim, element, ranges, style)
        mapping['fill_color'] = {'field': cdim.name,
                                 'transform': cmapper}

        return data, mapping

    @staticmethod
    def get_hex_data(element, gridsize, agg_func):
        print("GRID_SZ =", gridsize )
        x, y = (element.dimension_values(i) for i in range(2))

        collection = plt.hexbin(x, y, gridsize=gridsize, reduce_C_function=agg_func)

        # coordinates for single hexagonal patch
        hx = [0, .5, .5, 0, -.5, -.5]
        hy = [-.5/np.cos(np.pi/6), -.5*np.tan(np.pi/6), .5*np.tan(np.pi/6), 
              .5/np.cos(np.pi/6), .5*np.tan(np.pi/6), -.5*np.tan(np.pi/6)]  

        # number of hexagons needed
        m = len(collection.get_offsets())

        # scale of hexagons
        n = (x.max() - x.min()) / gridsize

        # y_scale to adjust for aspect ratio
        y_scale = (y.max() - y.min())/(x.max() - x.min())

        # coordinates for all hexagonal patches
        agg = collection.get_array()
        hxs = np.array([hx]*m)*n + np.vstack(collection.get_offsets()[:,0])
        hys = np.array([hy]*m)*n*y_scale + np.vstack(collection.get_offsets()[:,1])
        return zip(*((hx, hy, val) for hx, hy, val in zip(hxs, hys, agg) if val > 0))  

# -----------------------------------------------------------------------------------
hv.Store.register({HexBins: HexBinPlot}, 'bokeh')

options = hv.Store.options(backend='bokeh')
options.HexBins = hv.Options('style', line_color='black')

In [9]:
%%opts HexBins [logz=True width=400 height=400 ] (fill_alpha=0.8 line_alpha=0)
HexBins(np.random.randn(100000,3) )(plot=dict( gridsize=10 )) *\
HexBins(np.random.randn(100000,3)+3)(style=dict( cmap='hot'), plot=dict(gridsize=40))

GRID_SZ = 10
GRID_SZ = 40


:Overlay
   .HexBins.I  :HexBins   [x,y]
   .HexBins.II :HexBins   [x,y]

In [11]:
%%opts WMTS [height=250 width=300 xaxis=None yaxis=None show_grid=True]
%%opts HexBins (alpha=0.3 cmap='hot')
# So only specify one dimension. Note zoom and pan works, as does reset
# Use a Mercator projection

# Austin, Texas
e1=lonlat_corners_to_extents([-97.782,30.199],[-97.699,30.300], to=ccrs.GOOGLE_MERCATOR);# print("e1 =", e1)
e2=lonlat_corners_to_extents([-97.772,30.199],[-97.699,30.300], to=ccrs.GOOGLE_MERCATOR);# print("e2 =", e2)
# Setting extent=e1 shows parts of Mexico

data = np.random.randn(100000,3)
data[:,0] = 800.*data[:,0] + 0.5*(e2[0]+e2[2])
data[:,1] = 300.*data[:,1] + 0.5*(e2[1]+e2[3])

gv.WMTS( tiles2['CartoDB'], extents=e2, crs=ccrs.GOOGLE_MERCATOR) *\
HexBins( data )(plot=dict(gridsize=5))

GRID_SZ = 5


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .HexBins.I :HexBins   [x,y]

##### Image Overlay <span style='color:red;'> TODO FIX</span>

In [ ]:
%%opts Image [colorbar=True fig_size=200] (cmap='viridis')

# xarray cannot read my local copy of ensemble.nc???
#import h5py
#ensemble = xr.DataArray(h5py.File('./sample-data/ensemble.nc','r'))
ensemble = xr.open_dataset('sample-data/ensemble.nc')

kdims = ['time', 'longitude', 'latitude']
vdims = ['surface_temperature']

dataset = gv.Dataset(ensemble, kdims=kdims, crs=ccrs.PlateCarree())
dataset.to(gv.Image, ['longitude', 'latitude'], ['surface_temperature'], ['time']) * gf.coastline()